In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from peft import LoraConfig, get_peft_model
import torch
import wandb
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
from datasets import load_metric

In [ ]:
# Initialize wandb for tracking
wandb.init(project="nepali-english-nmt")

In [ ]:
# Load the dataset
data_path = "./cleaned_data.csv"  
df = pd.read_csv(data_path)

In [ ]:
# Split the data into train, validation, and test sets
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [ ]:
# Convert pandas DataFrames to Hugging Face datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Load tokenizer and model
model_name = "facebook/mbart-large-50"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Set the correct language codes for mBART
src_lang = "ne_NP"
tgt_lang = "en_XX"

tokenizer.src_lang = src_lang
tokenizer.tgt_lang = tgt_lang


In [ ]:
# Tokenizer function for Nepali to English
def preprocess_function(examples):
    inputs = [ex for ex in examples["Nepali"]]
    targets = [ex for ex in examples["English"]]
    
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

# Load LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],  # Choose appropriate modules
    bias="none"
)

model = get_peft_model(model, lora_config)

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    report_to="wandb",  # Enable wandb for logging
)

In [ ]:
# Define the Trainer with Hugging Face’s Trainer API
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
metric = load_metric("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in preds]
    labels = [[tokenizer.decode(label, skip_special_tokens=True)] for label in labels]
    return {"bleu": metric.compute(predictions=preds, references=labels)["score"]}


In [ ]:
trainer.evaluate(eval_dataset=val_dataset, metric_key_prefix="eval")

In [ ]:
# Push model to Hugging Face Hub
model.push_to_hub("nepali-to-english-mbart-lora")

print("Model training, evaluation, and push to Hugging Face Hub complete!")